In [ ]:
train = read.csv(file = '../input/457-engineered/train_engineered (2).csv', header = TRUE)
test = read.csv(file = '../input/457-engineered/test_engineered (2).csv', header = TRUE)
train_sub = train[train$Sales > 0, ]#only use positive Sales in training data
testOpen = (test$Open == 1)
test_sub = test[testOpen, ] #only needs to do prediction for open days
colnames(train_sub)

from feature selection, our current models and feature selected are:

Customer:

lm(formula = Customers ~ factor(Store) + factor(DayOfWeek) + Promo + SchoolHoliday + factor(year) + factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):Promo + factor(Store):SchoolHoliday + factor(Store):factor(year) + factor(Store):factor(month) + factor(Store):day + factor(DayOfWeek):SchoolHoliday + factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + factor(DayOfWeek):day + Promo:factor(year) + Promo:factor(month) + Promo:day + SchoolHoliday:factor(month) + SchoolHoliday:day + factor(month):day + CompetitionDistance + StoreType, data = train_sub)

Sales:

lm(formula = Sales ~ factor(Store) + factor(DayOfWeek) + Customers + Promo + SchoolHoliday + factor(year) + factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):Customers + factor(Store):Promo + factor(Store):SchoolHoliday + factor(Store):factor(year) + factor(Store):factor(month) + factor(Store):day + factor(DayOfWeek):Customers + factor(DayOfWeek):Promo + factor(DayOfWeek):SchoolHoliday + factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + Customers:Promo + Customers:SchoolHoliday + Customers:factor(year) + Customers:factor(month) + Customers:day + Promo:SchoolHoliday + Promo:factor(year) + Promo:factor(month) + Promo:day + SchoolHoliday:factor(year) + SchoolHoliday:factor(month) + factor(year):factor(month) + factor(year):day + factor(month):day + CompetitionDistance, data = train_sub)

# Lasso regression for sales

In [ ]:
library(caret)
dummies <- dummyVars(formula = Sales ~ factor(Store) + factor(DayOfWeek) + Customers + Promo + SchoolHoliday + factor(year) + factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):Customers + factor(Store):Promo + factor(Store):SchoolHoliday + factor(Store):factor(year) + factor(Store):factor(month) + factor(Store):day + factor(DayOfWeek):Customers + factor(DayOfWeek):Promo + factor(DayOfWeek):SchoolHoliday + factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + Customers:Promo + Customers:SchoolHoliday + Customers:factor(year) + Customers:factor(month) + Customers:day + Promo:SchoolHoliday + Promo:factor(year) + Promo:factor(month) + Promo:day + SchoolHoliday:factor(year) + SchoolHoliday:factor(month) + factor(year):factor(month) + factor(year):day + factor(month):day + CompetitionDistance, data = train_sub)
train_dummies = predict(dummies, newdata = train_sub)
dim(train_dummies)


In [ ]:
library(glmnet)
x = as.matrix(train_dummies)
y_train = train_sub$Sales
myridge = glmnet(x, y_train,alpha = 1, standardize = TRUE)


In [ ]:
lam.seq = exp(seq(-5, 3, length=100))
lasso.cv.out = cv.glmnet(x,y_train, alpha = 1, standardize = TRUE, lambda = lam.seq) 
plot(lasso.cv.out)

In [ ]:
# smallest cv error
lasso.cv.out$lambda.min

In [ ]:
# one-standard deviation rule
lasso.cv.out$lambda.1se

# TEST RMSPE

In [ ]:
library(MLmetrics)
# self test RMSPE
smp_size <- floor(0.75 * nrow(train_sub))

## set the seed to make your partition reproducible
set.seed(123)
train_ind <- sample(seq_len(nrow(train_sub)), size = smp_size)

train_1 <- train_sub[train_ind, ]
train_1dummy <-as.matrix(predict(dummies, newdata = train_1))
test_1 <- train_sub[-train_ind, ]
test_1dummy <-as.matrix(predict(dummies, newdata = test_1))

# lambda = lasso.cv.out$lambda.min 0.0696126264688813
# lambda = lasso.cv.out$lambda.1se 0.0699129321153788
lasso.mod <- glmnet(train_1dummy, train_1$Sales,alpha = 1, standardize = TRUE, lambda = lasso.cv.out$lambda.min)
y_pred = predict(lasso.mod, test_1dummy)
y_true = as.numeric(as.vector(unlist(test_1['Sales'])))

RMSPE(y_pred = y_pred, y_true = y_true)

lambda.min a little bit better

# Lasso regression for Customer

In [ ]:
customer_dum <- dummyVars(formula = Customers ~ factor(Store) + factor(DayOfWeek) + Promo + SchoolHoliday + factor(year) + factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):Promo + factor(Store):SchoolHoliday + factor(Store):factor(year) + factor(Store):factor(month) + factor(Store):day + factor(DayOfWeek):SchoolHoliday + factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + factor(DayOfWeek):day + Promo:factor(year) + Promo:factor(month) + Promo:day + SchoolHoliday:factor(month) + SchoolHoliday:day + factor(month):day + CompetitionDistance + StoreType, data = train_sub)

train_c_dum = predict(customer_dum, newdata = train_sub)
dim(train_c_dum)

In [ ]:
x_c = as.matrix(train_c_dum)
y_c_train = train_sub$Customers
mylasso = glmnet(x_c, y_c_train, alpha = 1,standardize = TRUE)

In [ ]:
lam.seq = exp(seq(-5, 3, length=100))
lasso.cv.out = cv.glmnet(x_c,y_c_train, alpha = 1, standardize = TRUE,lambda = lam.seq) 
plot(lasso.cv.out)

In [ ]:
# smallest cv error
lasso.cv.out$lambda.min

In [ ]:
# one-standard deviation rule
lasso.cv.out$lambda.1se

In [ ]:
library(MLmetrics)
# self test RMSPE
smp_size <- floor(0.75 * nrow(train_sub))

## set the seed to make your partition reproducible
set.seed(123)
train_ind <- sample(seq_len(nrow(train_sub)), size = smp_size)

train_1 <- train_sub[train_ind, ]
train_1dummy <-as.matrix(predict(customer_dum, newdata = train_1))
test_1 <- train_sub[-train_ind, ]
test_1dummy <-as.matrix(predict(customer_dum, newdata = test_1))

# lambda = ridge.cv.out$lambda.min 0.148203771003941
# lambda = ridge.cv.out$lambda.1se 0.15686046057034
lasso.mod <- glmnet(train_1dummy, train_1$Customers, standardize = TRUE,alpha = 1, lambda = lasso.cv.out$lambda.1se)
y_pred = predict(lasso.mod, test_1dummy)
y_true = as.numeric(as.vector(unlist(test_1['Customers'])))

RMSPE(y_pred = y_pred, y_true = y_true)

0.0695911593683118, 0.15686046057034

Actually customer one is worse than linear regression, but sales one is better.